In [5]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [6]:
import pandas as pd

In [7]:
files = [
    './results/results-2023-08-3-layer-512-sum-ackley_3d-0-particle-leaky_reluhigh-dimensional.parquet',
    './results/results-2023-08-3-layer-512-sum-ackley_5d-0-particle-leaky_reluhigh-dimensional.parquet',
    './results/results-2023-08-3-layer-512-sum-ackley_30d-0-particle-leaky_reluhigh-dimensional.parquet',
    './results/results-2023-08-3-layer-512-sum-ackley_100d-0-particle-leaky_reluhigh-dimensional.parquet',
]

In [9]:
df = pd.read_parquet(files)

In [10]:
df

,x1,x2,x3,f,algorithm,total_time,input_size,hidden_size,hidden_activation,output_activation,agg_function,problem_name,global_minimum,dimensions
0,-1.093596e-07,-1.086391e-08,5.730915e-08,2.862346e-07,Deeplifting,119.070853,512,512-512-512,sine,leaky_relu,sum,ackley_3d,0.0,3
1,-2.362199e-10,-6.691891e-10,9.578308e-09,2.315031e-08,Deeplifting,995.560220,512,512-512-512,sine,leaky_relu,sum,ackley_5d,0.0,5
2,1.844665e-07,-1.427955e-07,-1.505370e-07,3.897032e-07,Deeplifting,5703.610024,512,512-512-512,sine,leaky_relu,sum,ackley_30d,0.0,30
3,-1.600911e-08,2.234648e-08,-7.853131e-09,1.255706e-07,Deeplifting,42608.341140,512,512-512-512,sine,leaky_relu,sum,ackley_100d,0.0,100


11